In [ ]:
%pip install scikit-learn imbalanced-learn==0.11.0
%pip install matplotlib
%pip install seaborn
%pip install mlflow dagshub
%pip install xgboost

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import warnings
import dagshub
import mlflow
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ieee-fraud-detection/sample_submission.csv
/kaggle/input/ieee-fraud-detection/test_identity.csv
/kaggle/input/ieee-fraud-detection/train_identity.csv
/kaggle/input/ieee-fraud-detection/test_transaction.csv
/kaggle/input/ieee-fraud-detection/train_transaction.csv


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 15)
pd.set_option('display.width', None)
pd.set_option('display.expand_frame_repr', False)

# to ignore warnings in cells
warnings.filterwarnings('ignore')

# Authorize on DagsHub

In [4]:
dagshub.init(repo_owner='b3tameche', repo_name='kaggle-fraud-detection', mlflow=True)

❗❗❗ AUTHORIZATION REQUIRED ❗❗❗



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=6d49fac4-0414-4096-ac0e-9fab06f1c0fa&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=b39f9fc80f3a9db9d746cf525d54b32405efcc870accd3b3d03ca1a1eb8708ac




Output()

Accessing as b3tameche

Initialized MLflow to track repo "b3tameche/kaggle-fraud-detection"

Repository b3tameche/kaggle-fraud-detection initialized!

# Loading the best model

In [5]:
best_model_name = "best_model_xgboost"
best_model_version = "2"

best_model_uri = f'models:/{best_model_name}/{best_model_version}'

best_model = mlflow.sklearn.load_model(best_model_uri)

# Submission

In [6]:
df_test_transaction = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_transaction.csv")
df_test_identity = pd.read_csv("/kaggle/input/ieee-fraud-detection/test_identity.csv")
df_test = pd.merge(df_test_transaction, df_test_identity, on="TransactionID", how="left")
df_test.drop('TransactionID', axis=1, inplace=True)

In [7]:
y_test_pred = best_model.predict(df_test)

final = pd.DataFrame({
  'TransactionID': np.arange(3663549, 3663549 + len(y_test_pred)),
  'isFraud': y_test_pred
})

final.to_csv('submission.csv', index=False)

print(final.head())

   TransactionID  isFraud
0        3663549        0
1        3663550        0
2        3663551        0
3        3663552        0
4        3663553        0
